## Problem Set 3 (Part 2 of 2): Interacting with the Alpha Vantage API using Python libraries

In Part 2 of Problem Set 3 we will be using the data retrived from the Alpha Vantage API and creating an optimized portfolio of stocks based on Modern Portfolio Theory. The PyPortfolioOpt package will be used to optimize a portfolio of stocks based maximizing the Sharpe Ratio. This exercise is an example of how a robo-advisor would use data APIs to continuously monitor and adjusts stock portfolios. 

This part of the exercise demonstrates:

1. Setting up a programming environment to work with APIs
2. Data preprocessing to create compatible datasets
3. Using a quantititive popular portfolio optimization library to estimate risks and weights

Documentation links:
<br />Alpha Vantage: https://www.alphavantage.co/documentation/
<br />Alpha Vantage Python library: https://alpha-vantage.readthedocs.io/en/latest/#
<br />Pandas DataFrame: https://pandas.pydata.org/pandas-docs/stable/reference/frame.html
<br />PyPortfolioOpt: https://pyportfolioopt.readthedocs.io/en/latest/index.html

In [1]:
api_key = "2YY86W5ODA9UN8ZY"

In [2]:
import pandas as pd
import requests
from alpha_vantage.timeseries import TimeSeries
import matplotlib.pyplot as plt

# PyPortfolioOpt libraries
from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import risk_models 
from pypfopt import expected_returns
from pypfopt.discrete_allocation import DiscreteAllocation, get_latest_prices

In [3]:
ts = TimeSeries(key=api_key,output_format='pandas')

In [4]:
fb_data, fb_meta_data = ts.get_daily(symbol='FB', outputsize='full')
msft_data, msft_meta_data = ts.get_daily(symbol='MSFT', outputsize='full')
aapl_data, aapl_meta_data = ts.get_daily(symbol='AAPL', outputsize='full')
amzn_data, amzn_meta_data = ts.get_daily(symbol='AMZN', outputsize='full')

Isolate "close" column of data for each security:

In [5]:
fb_clean = fb_data['4. close']
msft_clean = msft_data['4. close']
aapl_clean = aapl_data['4. close']
amzn_clean = amzn_data['4. close']

Create new dataframe with tickers as columns:

In [6]:
df = pd.DataFrame(columns=['fb', 'msft', 'aapl', 'amzn'])

In [7]:
df

,fb,msft,aapl,amzn


Place data into new dataframe:

In [8]:
df['fb'] = fb_clean
df['msft'] = msft_clean
df['aapl'] = aapl_clean
df['amzn'] = amzn_clean
print(df)

                 fb     msft     aapl      amzn
date                                           
2012-05-18   38.232   29.270  530.380   213.850
2012-05-21   34.030   29.750  561.280   218.110
2012-05-22   31.000   29.760  556.970   215.330
2012-05-23   32.000   29.110  570.560   217.280
2012-05-24   33.030   29.070  565.320   215.240
2012-05-25   31.910   29.060  562.290   212.890
2012-05-29   28.840   29.560  572.270   214.750
2012-05-30   28.190   29.340  579.170   209.230
2012-05-31   29.600   29.190  577.730   212.910
2012-06-01   27.720   28.450  560.990   208.220
2012-06-04   26.900   28.550  564.290   214.570
2012-06-05   25.869   28.505  562.830   213.210
2012-06-06   26.810   29.350  571.460   217.640
2012-06-07   26.310   29.230  571.720   218.800
2012-06-08   27.100   29.650  580.320   218.480
2012-06-11   27.005   28.895  571.170   216.500
2012-06-12   27.400   29.290  576.160   216.420
2012-06-13   27.270   29.130  572.160   214.730
2012-06-14   28.290   29.340  571.530   

Take only the earlist half of observations:

In [9]:
df = df[0:924]

Calculate expected returns:

In [10]:
print(df)

                 fb    msft     aapl     amzn
date                                         
2012-05-18   38.232  29.270  530.380  213.850
2012-05-21   34.030  29.750  561.280  218.110
2012-05-22   31.000  29.760  556.970  215.330
2012-05-23   32.000  29.110  570.560  217.280
2012-05-24   33.030  29.070  565.320  215.240
2012-05-25   31.910  29.060  562.290  212.890
2012-05-29   28.840  29.560  572.270  214.750
2012-05-30   28.190  29.340  579.170  209.230
2012-05-31   29.600  29.190  577.730  212.910
2012-06-01   27.720  28.450  560.990  208.220
2012-06-04   26.900  28.550  564.290  214.570
2012-06-05   25.869  28.505  562.830  213.210
2012-06-06   26.810  29.350  571.460  217.640
2012-06-07   26.310  29.230  571.720  218.800
2012-06-08   27.100  29.650  580.320  218.480
2012-06-11   27.005  28.895  571.170  216.500
2012-06-12   27.400  29.290  576.160  216.420
2012-06-13   27.270  29.130  572.160  214.730
2012-06-14   28.290  29.340  571.530  214.450
2012-06-15   30.014  30.020  574.1

In [11]:
mu = expected_returns.mean_historical_return(df)

Calculate sample covariance matrix:

In [12]:
s = risk_models.sample_cov(df)

In [13]:
print(s)

            fb      msft      aapl      amzn
fb    0.183263  0.018451  0.020780  0.037626
msft  0.018451  0.057307  0.020826  0.026703
aapl  0.020780  0.020826  0.269413  0.020855
amzn  0.037626  0.026703  0.020855  0.093943


Optimize the weights to maximzie the Sharpe Ratio:

In [14]:
ef = EfficientFrontier(mu,s)
weights = ef.max_sharpe()
ef.portfolio_performance(verbose=True)

Expected annual return: 28.3%
Annual volatility: 23.5%
Sharpe Ratio: 1.12


(0.28315300039734614, 0.2353086186227769, 1.1183313298830186)

In [15]:
print(weights)

{'fb': 0.23331629887166944, 'msft': 0.2728212151853775, 'aapl': 9.226560487851643e-17, 'amzn': 0.49386248594295307}


Using optimized weights, compute portfolio accolation:

In [16]:
latest_prices = get_latest_prices(df)
da = DiscreteAllocation(weights, latest_prices, total_portfolio_value=1000000)
allocation, leftover = da.greedy_portfolio()
print(allocation)

1 out of 4 tickers were removed
{'amzn': 859, 'msft': 5404, 'fb': 2477}


To update portfolio, call the API again for latest data and run optimization model again:

In [17]:
fb_data, fb_meta_data = ts.get_daily(symbol='FB', outputsize='full')
msft_data, msft_meta_data = ts.get_daily(symbol='MSFT', outputsize='full')
aapl_data, aapl_meta_data = ts.get_daily(symbol='AAPL', outputsize='full')
amzn_data, amzn_meta_data = ts.get_daily(symbol='AMZN', outputsize='full')

In [18]:
df = pd.DataFrame(columns=['fb', 'msft', 'aapl', 'amzn'])

In [19]:
fb_clean = fb_data['4. close']
msft_clean = msft_data['4. close']
aapl_clean = aapl_data['4. close']
amzn_clean = amzn_data['4. close']

In [20]:
df['fb'] = fb_clean
df['msft'] = msft_clean
df['aapl'] = aapl_clean
df['amzn'] = amzn_clean
print(df)

                 fb     msft      aapl      amzn
date                                            
2012-05-18   38.232   29.270  530.3800   213.850
2012-05-21   34.030   29.750  561.2800   218.110
2012-05-22   31.000   29.760  556.9700   215.330
2012-05-23   32.000   29.110  570.5600   217.280
2012-05-24   33.030   29.070  565.3200   215.240
2012-05-25   31.910   29.060  562.2900   212.890
2012-05-29   28.840   29.560  572.2700   214.750
2012-05-30   28.190   29.340  579.1700   209.230
2012-05-31   29.600   29.190  577.7300   212.910
2012-06-01   27.720   28.450  560.9900   208.220
2012-06-04   26.900   28.550  564.2900   214.570
2012-06-05   25.869   28.505  562.8300   213.210
2012-06-06   26.810   29.350  571.4600   217.640
2012-06-07   26.310   29.230  571.7200   218.800
2012-06-08   27.100   29.650  580.3200   218.480
2012-06-11   27.005   28.895  571.1700   216.500
2012-06-12   27.400   29.290  576.1600   216.420
2012-06-13   27.270   29.130  572.1600   214.730
2012-06-14   28.290 

In [21]:
mu = expected_returns.mean_historical_return(df)
s = risk_models.sample_cov(df)
ef = EfficientFrontier(mu,s)
weights =ef.max_sharpe()
ef.portfolio_performance(verbose=True)
print(weights)

Expected annual return: 28.2%
Annual volatility: 22.3%
Sharpe Ratio: 1.17
{'fb': 0.13842025872784702, 'msft': 0.4283624365214419, 'aapl': 1.431146867680866e-17, 'amzn': 0.4332173047507112}


In [22]:
latest_prices = get_latest_prices(df)
da = DiscreteAllocation(weights, latest_prices, total_portfolio_value=1000000)
allocation, leftover = da.greedy_portfolio()
print(allocation)

1 out of 4 tickers were removed
{'amzn': 251, 'msft': 3104, 'fb': 780}
